In [26]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os
import sys
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
sys.path.append(os.getenv('scraper_path'))
from YT_Scrapy import YtScraper
scrapper = YtScraper()

In [13]:
# def scroll_down(driver):
#     last_height = driver.execute_script("return window.scrollY;")
#     while True:
#         driver.find_element("tag name", "body").send_keys(Keys.END)
#         # driver.implicitly_wait(20)
#         time.sleep(5)
#         new_height = driver.execute_script("return window.scrollY;")
        
#         if new_height == last_height:
#             # driver.find_element("tag name", "body").send_keys(Keys.END)
#             break
#         last_height = new_height

def get_video_id(driver,url:str):
    def scroll_down(driver):
        last_height = driver.execute_script("return window.scrollY;")
        while True:
            driver.find_element("tag name", "body").send_keys(Keys.END)
            # driver.implicitly_wait(20)
            time.sleep(5)
            new_height = driver.execute_script("return window.scrollY;")
            
            if new_height == last_height:
                # driver.find_element("tag name", "body").send_keys(Keys.END)
                break
            last_height = new_height
    live_url = url+'/streams'
    shorts_url = url+'/shorts'
    vod_url = url+'/videos'
    driver.get(live_url)
    driver.implicitly_wait(15)
    scroll_down(driver)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # tmp = soup.find('div', {'id':'contents','class':'style-scope ytd-rich-grid-renderer'})
    data = soup.find_all('a',{'id':'thumbnail','class':'yt-simple-endpoint inline-block style-scope ytd-thumbnail'})
    live_video_id = []
    for datas in data:
        href = datas.get('href')
        if href:
            live_video_id.append(href.removeprefix('/watch?v='))
    # del soup,data
    driver.get(shorts_url)
    driver.implicitly_wait(15)
    scroll_down(driver)
    soup = BeautifulSoup(driver.page_source, 'html.parser') 
    # tmp = soup.find('div', {'id':'contents','class':'style-scope ytd-rich-grid-renderer'})
    data = soup.find_all('a', {'class':'ShortsLockupViewModelHostEndpoint reel-item-endpoint'})
    shorts_id = []
    for datas in data:
        href = datas.get('href')
        if href:
            shorts_id.append(href.removeprefix('/shorts/'))
    # del soup,data
    driver.get(vod_url)
    scroll_down(driver)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    data = soup.find_all('a',{'id':'thumbnail','class':'yt-simple-endpoint inline-block style-scope ytd-thumbnail'})
    vod_id = []
    for datas in data:
        href = datas.get('href')
        if href:
            vod_id.append(href.removeprefix('/watch?v='))
    final_data = {
        'live':live_video_id,
        'shorts':shorts_id,
        'vod':vod_id
    }
    del live_video_id,shorts_id,vod_id
    return final_data

In [28]:
# Driver Options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

PATH = "C:/Program Files/chromedriver.exe"
SERVICE = Service(PATH)
DRIVER = webdriver.Chrome(service=SERVICE,options=chrome_options)
YT_ID = '@cecilialieberia'
URL = 'http://www.youtube.com/'+YT_ID
video_ids = get_video_id(url=URL,driver=DRIVER)

In [29]:
video_ids.keys()

dict_keys(['live', 'shorts', 'vod'])

In [34]:
ch_data = scrapper.scrape_channel_basic_data(YT_ID)

In [38]:
ch_data['date_created(UTC)'] = str(ch_data['date_created(UTC)'])

In [40]:
with open('data/ch_data.json', 'w') as json_file:
    json.dump(ch_data, json_file, indent=4)

In [42]:
for key in video_ids.keys():
    video_data = scrapper.scrape_video_data(video_ids.get(key))
    video_data['video_type'] = key
    video_data.to_csv(f'data/data_{key}.csv',index=False)